In [ ]:

!pip install keras-tuner
!pip install backtesting

In [ ]:

import datetime as dt
import yfinance as yf
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch


In [ ]:

train_start_date = dt.datetime(2022, 1, 1)
train_end_date = dt.datetime(2023, 12, 31)

test_start_date = dt.datetime(2024, 1, 1)
test_end_date = dt.datetime(2024, 6, 5)

train_data = yf.download("AAPL", start=train_start_date, end=train_end_date)
test_data = yf.download("AAPL", start=test_start_date, end=test_end_date)

train_data = train_data.iloc[:, :1]
test_data = test_data.iloc[:, :1]


dataset_train = train_data.Open.values
dataset_test = test_data.Open.values


dataset_train = np.reshape(dataset_train, (-1, 1))
dataset_test = np.reshape(dataset_test, (-1, 1))

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)


X_train = np.array([scaled_train[i-1:i] for i in range(1, len(scaled_train))])
y_train = scaled_train[1:]


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))


regressor = Sequential()

regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50))
regressor.add(Dropout(0.2))


regressor.add(Dense(units=1, activation='sigmoid'))

regressor.compile(optimizer='adam', loss="mean_squared_error")


regressor.fit(X_train, y_train, epochs=20, batch_size=2)

class RNNForecastStrategy(Strategy):
    def init(self):
        self.signal = self.I(self.predict_prices, self.data.Open)

    def predict_prices(self, data):

        data = np.array(data).reshape(-1, 1)


        scaled_data = scaler.transform(data)
        X = np.array([scaled_data[i-1:i] for i in range(1, len(scaled_data))])
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))


        predicted = regressor.predict(X)
        predicted = scaler.inverse_transform(predicted)


        predicted_padded = np.zeros(len(data))
        predicted_padded[1:] = predicted.flatten()
        predicted_padded[0] = data[0]

        return predicted_padded

    def next(self):
        if crossover(self.signal, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, self.signal):
            self.sell()

data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, RNNForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()


In [ ]:
data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, RNNForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()

In [ ]:
import datetime as dt
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.optimizers import Adam
from keras_tuner import RandomSearch, HyperParameters
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

train_start_date = dt.datetime(2023, 1, 1)
train_end_date = dt.datetime(2023, 12, 31)
test_start_date = dt.datetime(2024, 1, 1)
test_end_date = dt.datetime(2024, 6, 5)

train_data = yf.download("AAPL", start=train_start_date, end=train_end_date)
test_data = yf.download("AAPL", start=test_start_date, end=test_end_date)

dataset_train = train_data['Open'].values.reshape(-1, 1)
dataset_test = test_data['Open'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

seq_length = 60
X_train, y_train = create_sequences(scaled_train, seq_length)
X_test, y_test = create_sequences(scaled_test, seq_length)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=50, max_value=200, step=50),
                   return_sequences=True,
                   input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(LSTM(units=hp.Int(f'units_{i}', min_value=50, max_value=200, step=50), return_sequences=True))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(LSTM(units=hp.Int('units_last', min_value=50, max_value=200, step=50)))
    model.add(Dropout(hp.Float('dropout_last', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(units=1))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    return model


tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='lstm_hyperparam_tuning',
    project_name='AAPL_LSTM'
)


X_train_partial = X_train[:int(0.8 * len(X_train))]
y_train_partial = y_train[:int(0.8 * len(y_train))]
X_val = X_train[int(0.8 * len(X_train)):]
y_val = y_train[int(0.8 * len(y_train)):]


tuner.search(X_train_partial, y_train_partial, epochs=10, validation_data=(X_val, y_val), batch_size=32)


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first LSTM layer is {best_hps.get('units')},
the optimal dropout rate is {best_hps.get('dropout')},
the optimal number of layers is {best_hps.get('num_layers')},
and the optimal learning rate is {best_hps.get('learning_rate')}.
""")

model = tuner.hypermodel.build(best_hps)


model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


class LSTMForecastStrategy(Strategy):
    def init(self):
        self.signal = self.I(self.predict_prices, self.data.Open)

    def predict_prices(self, data):

        data = np.array(data).reshape(-1, 1)

        scaled_data = scaler.transform(data)
        X = np.array([scaled_data[i-seq_length:i] for i in range(seq_length, len(scaled_data))])
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))


        if X.size > 0:
            predicted = model.predict(X)
            predicted = scaler.inverse_transform(predicted)

            predicted_padded = np.zeros(len(data))
            predicted_padded[seq_length:] = predicted.flatten()
            predicted_padded[:seq_length] = data[:seq_length].flatten()


            return predicted_padded
        else:
            return np.zeros(len(data))

    def next(self):
        if crossover(self.signal, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, self.signal):
            self.sell()


data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, LSTMForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
print(stats)


Trial 2 Complete [00h 01m 24s]
val_loss: 0.003505009226500988

Best val_loss So Far: 0.003505009226500988
Total elapsed time: 00h 03m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
200               |100               |units
0.4               |0.2               |dropout
2                 |2                 |num_layers
150               |100               |units_0
0.2               |0.2               |dropout_0
100               |150               |units_last
0.4               |0.4               |dropout_last
0.001             |0.0001            |learning_rate
50                |200               |units_1
0.3               |0.2               |dropout_1
100               |100               |units_2
0.4               |0.4               |dropout_2
150               |50                |units_3
0.3               |0.2               |dropout_3

Epoch 1/10
5/5 [==============================] - 12s 719ms/step - loss: 0.2584 - val_loss: 0.0071
Epoch 2/10
5/5 